# Task5 变形

## 1 知识梳理（重点记忆）

In [2]:
import pandas as pd
import numpy as np

## 2 练一练

### 2.1 第1题
在上面的边际汇总例子中，行或列的汇总为新表中行元素或者列元素的平均值，而总体的汇总为新表中四个元素的平均值。这种关系一定成立吗？若不成立，请给出一个例子来说明。

**我的解答：**

In [3]:
df = pd.DataFrame({'Name':['San Zhang', 'San Zhang', 
                              'San Zhang', 'San Zhang',
                              'Si Li', 'Si Li', 'Si Li', 'Si Li'],
                   'Subject':['Chinese', 'Chinese', 'Math', 'Math',
                                 'Chinese', 'Chinese', 'Math', 'Math'],
                   'Grade':[80, 90, 100, 90, 70, 80, 85, 95]})
df

,Name,Subject,Grade
0,San Zhang,Chinese,80
1,San Zhang,Chinese,90
2,San Zhang,Math,100
3,San Zhang,Math,90
4,Si Li,Chinese,70
5,Si Li,Chinese,80
6,Si Li,Math,85
7,Si Li,Math,95


In [6]:
df.pivot_table(index = 'Name',
               columns = 'Subject',
               values = 'Grade',
               aggfunc=lambda x:x.min(),
               margins=True)

Subject,Chinese,Math,All
Name,,,
San Zhang,80,90,80
Si Li,70,85,70
All,70,85,70


上面的说法不正确，总体的汇总为应该为所有Grade的值进行聚合，而不是新表中四个元素，通过查看pandas源码，`pivot.py`下的`_compute_grand_margin`函数，描述了整个计算过程：  
- 遍历data[values]
- 根据给定的函数，计算grand_margin[k] = aggfunc(v)
- 最后返回计算得到的grand_margin

```python
def _compute_grand_margin(data, values, aggfunc, margins_name: str = "All"):
    if values:
        grand_margin = {}
        for k, v in data[values].items():
            try:
                if isinstance(aggfunc, str):
                    grand_margin[k] = getattr(v, aggfunc)()
                elif isinstance(aggfunc, dict):
                    if isinstance(aggfunc[k], str):
                        grand_margin[k] = getattr(v, aggfunc[k])()
                    else:
                        grand_margin[k] = aggfunc[k](v)
                else:
                    grand_margin[k] = aggfunc(v)
            except TypeError:
                pass
        return grand_margin
    else:
        return {margins_name: aggfunc(data.index)}
```

### 2.2 第2题
前面提到了`crosstab`的性能劣于`pivot_table`，请选用多个聚合方法进行验证。

**我的解答：**

In [7]:
df = pd.read_csv('../data/learn_pandas.csv')
df.head()

,School,Grade,Name,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
0,Shanghai Jiao Tong University,Freshman,Gaopeng Yang,Female,158.9,46.0,N,1,2019/10/5,0:04:34
1,Peking University,Freshman,Changqiang You,Male,166.5,70.0,N,1,2019/9/4,0:04:20
2,Shanghai Jiao Tong University,Senior,Mei Sun,Male,188.9,89.0,N,2,2019/9/12,0:05:22
3,Fudan University,Sophomore,Xiaojuan Sun,Female,NaN,41.0,N,2,2020/1/3,0:04:08
4,Fudan University,Sophomore,Gaojuan You,Male,174.0,74.0,N,2,2019/11/6,0:05:22


1. `mean`聚合函数，统计身高`Height`的均值

In [25]:
%timeit -n 30 pd.crosstab(index = df.School, columns = df.Transfer, values = df.Height, aggfunc = 'mean')

30 loops, best of 5: 7.32 ms per loop


In [26]:
%timeit -n 30 df.pivot_table(index = 'School', columns = 'Transfer', values = 'Height', aggfunc = 'mean')

30 loops, best of 5: 6.76 ms per loop


2. `max`聚合函数，统计体重`Weight`的最大值

In [28]:
%timeit -n 30 pd.crosstab(index = df.School, columns = df.Transfer, values = df.Weight, aggfunc = 'max')

30 loops, best of 5: 7.2 ms per loop


In [29]:
%timeit -n 30 df.pivot_table(index = 'School', columns = 'Transfer', values = 'Weight', aggfunc = 'max')

30 loops, best of 5: 6.68 ms per loop


## 3 练习
### 3.1 Ex1：美国非法药物数据集

现有一份关于美国非法药物的数据集，其中`SubstanceName, DrugReports`分别指药物名称和报告数量：

In [38]:
df = pd.read_csv('../data/drugs.csv').sort_values(['State','COUNTY','SubstanceName'],ignore_index=True)
df.head(3)

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2011,KY,ADAIR,Buprenorphine,3
1,2012,KY,ADAIR,Buprenorphine,5
2,2013,KY,ADAIR,Buprenorphine,4


1. 将数据转为如下的形式：

<img src="../source/_static/Ex5_1.png" width="35%">

2. 将第1问中的结果恢复为原表。
3. 按`State`分别统计每年的报告数量总和，其中`State, YYYY`分别为列索引和行索引，要求分别使用`pivot_table`函数与`groupby+unstack`两种不同的策略实现，并体会它们之间的联系。

**我的解答：**  

**第1问：**

In [49]:
res = df.pivot(index=['State', 'COUNTY', 'SubstanceName'], columns='YYYY', values='DrugReports')
res = res.reset_index().rename_axis(columns={'YYYY':''})
res.head()

,State,COUNTY,SubstanceName,2010,2011,2012,2013,2014,2015,2016,2017
0,KY,ADAIR,Buprenorphine,NaN,3.0,5.0,4.0,27.0,5.0,7.0,10.0
1,KY,ADAIR,Codeine,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
2,KY,ADAIR,Fentanyl,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,KY,ADAIR,Heroin,NaN,NaN,1.0,2.0,NaN,1.0,NaN,2.0
4,KY,ADAIR,Hydrocodone,6.0,9.0,10.0,10.0,9.0,7.0,11.0,3.0


**第2问：**

### 3.2 Ex2：特殊的wide_to_long方法

从功能上看，`melt`方法应当属于`wide_to_long`的一种特殊情况，即`stubnames`只有一类。请使用`wide_to_long`生成`melt`一节中的`df_melted`。（提示：对列名增加适当的前缀）

In [37]:
df = pd.DataFrame({'Class':[1,2],
                   'Name':['San Zhang', 'Si Li'],
                   'Chinese':[80, 90],
                   'Math':[80, 75]})
df

,Class,Name,Chinese,Math
0,1,San Zhang,80,80
1,2,Si Li,90,75
